In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, concatenate
from tensorflow.keras.models import Model

# 假设你有一个包含wide和deep特征的数据集
# 请替换下面的伪造数据为你自己的数据
wide_features = tf.random.normal((1000, 3))
deep_features = {
    'feature4': tf.random.uniform((1000, 1), maxval=10, dtype=tf.int32),
    'feature5': tf.random.uniform((1000, 1), maxval=20, dtype=tf.int32),
    'feature6': tf.random.uniform((1000, 1), maxval=15, dtype=tf.int32)
}

labels = tf.random.uniform((1000, 1), maxval=2, dtype=tf.int32)

# 定义 wide 特征的输入
wide_inputs = Input(shape=(3,), name='wide_input')

# 定义 deep 特征的输入
deep_inputs = {feature: Input(shape=(1,), name=feature) for feature in ['feature4', 'feature5', 'feature6']}

# 定义 wide 部分
wide_branch = wide_inputs

# 定义 deep 部分
embeddings = [Embedding(input_dim=deep_features[feature].numpy().max() + 1, output_dim=4)(deep_inputs[feature]) for feature in ['feature4', 'feature5', 'feature6']]
deep_branch = concatenate(embeddings)
deep_branch = tf.keras.layers.Flatten()(deep_branch)
deep_branch = Dense(64, activation='relu')(deep_branch)
deep_branch = Dense(32, activation='relu')(deep_branch)

# 合并 wide 和 deep 部分
combined = concatenate([wide_branch, deep_branch])

# 输出层
output = Dense(1, activation='sigmoid')(combined)

# 编译模型
model = Model(inputs=[wide_inputs] + list(deep_inputs.values()), outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 打印模型结构
model.summary()

# 训练模型
model.fit([wide_features] + list(deep_features.values()), labels, epochs=5, batch_size=32, validation_split=0.2)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 feature4 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 feature5 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 feature6 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 1, 4)         40          ['feature4[0][0]']               
                                                                                            

您提供的代码是 TensorFlow 的一个很好的例子，用于构建和训练一个 Wide & Deep 模型。这段代码清晰地展示了如何分别处理 Wide 特征（线性模型部分）和 Deep 特征（嵌入和深度神经网络部分），并将它们结合起来进行训练。

这里有几个关键点值得注意：

1. **特征处理**：您使用了嵌入层（`Embedding`）来处理分类特征。这是一种常见的方法，用于将类别特征转换为可以被深度学习模型处理的密集向量。

2. **模型结构**：Wide 部分直接使用原始特征，而 Deep 部分则通过几个隐藏层进行处理。这符合 Wide & Deep 模型的典型结构。

3. **合并层**：使用 `concatenate` 将 Wide 和 Deep 部分结合起来，是构建此类模型的常规做法。

4. **模型编译和训练**：您使用了 `adam` 优化器和二元交叉熵损失函数来编译模型，并对模型进行了训练。

5. **模型概览**：通过调用 `model.summary()`，您可以查看模型的结构，这有助于理解各层是如何堆叠在一起的。

6. **训练**：您使用了自动生成的数据来训练模型。在实际应用中，您将使用实际的数据集。

您的代码是一个很好的起点。根据实际应用的需求，您可能需要对模型的架构、特征处理、训练参数等进行进一步的调整和优化。例如，您可以尝试添加更多的隐藏层，调整嵌入层的输出维度，或者尝试不同的激活函数和优化器。此外，对于实际应用中的数据，您可能还需要进行更复杂的特征工程和预处理。

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 设置随机种子
np.random.seed(0)
tf.random.set_seed(0)

# 生成模拟数据
num_samples = 10000  # 样本数量

# Wide 特征（数值特征）
wide_features = np.random.rand(num_samples, 3)  # 假设有3个数值特征

# Deep 特征（分类特征）
deep_features = {
    'feature4': np.random.randint(0, 10, num_samples),  # 假设有10个不同的类别
    'feature5': np.random.randint(0, 20, num_samples),  # 假设有20个不同的类别
    'feature6': np.random.randint(0, 15, num_samples)   # 假设有15个不同的类别
}

# 目标变量（二元）
labels = np.random.randint(0, 2, num_samples)

# 转换为 pandas DataFrame 和 Series
wide_features = pd.DataFrame(wide_features, columns=['wide1', 'wide2', 'wide3'])
deep_features = pd.DataFrame(deep_features)
labels = pd.Series(labels)

# 划分训练集和验证集
wide_features_train, wide_features_val, deep_features_train, deep_features_val, labels_train, labels_val = train_test_split(
    wide_features, deep_features, labels, test_size=0.2, random_state=0)

# 将数据转换为 TensorFlow 数据集
def make_dataset(wide, deep, labels, batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(({"wide_input": wide, "feature4": deep['feature4'], "feature5": deep['feature5'], "feature6": deep['feature6']}, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(labels))
    dataset = dataset.batch(batch_size)
    return dataset

train_ds = make_dataset(wide_features_train, deep_features_train, labels_train)
val_ds = make_dataset(wide_features_val, deep_features_val, labels_val, shuffle=False)


In [7]:
# 训练模型
history = model.fit(
    train_ds,
    epochs=10,            # 指定训练的轮数
    validation_data=val_ds
)

# 评估模型
loss, accuracy = model.evaluate(val_ds)
print("Validation accuracy:", accuracy)


Epoch 1/10
250/250 [==============================] - 2s 3ms/step - loss: 0.6946 - accuracy: 0.5059 - val_loss: 0.6936 - val_accuracy: 0.4970
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6921 - accuracy: 0.5215 - val_loss: 0.6949 - val_accuracy: 0.5040
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6914 - accuracy: 0.5264 - val_loss: 0.6956 - val_accuracy: 0.5020
Epoch 4/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6904 - accuracy: 0.5354 - val_loss: 0.6966 - val_accuracy: 0.4965
Epoch 5/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6898 - accuracy: 0.5356 - val_loss: 0.6958 - val_accuracy: 0.5005
Epoch 6/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6881 - accuracy: 0.5421 - val_loss: 0.6970 - val_accuracy: 0.4965
Epoch 7/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6876 - accuracy: 0.5458 - val_loss: 0.6991 - val_accuracy: 0.5025
Epoch 

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 设置随机种子
np.random.seed(0)
tf.random.set_seed(0)

# 生成模拟数据
num_samples = 10000  # 样本数量

# Wide 特征（数值特征）
# 假设原来有3个特征，现在添加两个新的特征，总共5个特征
wide_features = np.random.rand(num_samples, 5)  # 现在有5个数值特征

# Deep 特征（分类特征）
deep_features = {
    'feature4': np.random.randint(0, 10, num_samples),
    'feature5': np.random.randint(0, 20, num_samples),
    'feature6': np.random.randint(0, 15, num_samples)
}

# 目标变量（二元）
labels = np.random.randint(0, 2, num_samples)

# 转换为 pandas DataFrame 和 Series
wide_features = pd.DataFrame(wide_features, columns=['wide1', 'wide2', 'wide3', 'wide4', 'wide5'])
deep_features = pd.DataFrame(deep_features)
labels = pd.Series(labels)

# 划分训练集和验证集
wide_features_train, wide_features_val, deep_features_train, deep_features_val, labels_train, labels_val = train_test_split(
    wide_features, deep_features, labels, test_size=0.2, random_state=0)

# 将数据转换为 TensorFlow 数据集
def make_dataset(wide, deep, labels, batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(({"wide_input": wide, "feature4": deep['feature4'], "feature5": deep['feature5'], "feature6": deep['feature6']}, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(labels))
    dataset = dataset.batch(batch_size)
    return dataset

train_ds = make_dataset(wide_features_train, deep_features_train, labels_train)
val_ds = make_dataset(wide_features_val, deep_features_val, labels_val, shuffle=False)


In [10]:
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, concatenate
from tensorflow.keras.models import Model

# 定义模型的输入
# Wide 特征的输入
wide_inputs = Input(shape=(5,), name='wide_input')

# Deep 特征的输入
feature4_input = Input(shape=(1,), name='feature4', dtype='int32')
feature5_input = Input(shape=(1,), name='feature5', dtype='int32')
feature6_input = Input(shape=(1,), name='feature6', dtype='int32')

# 构建 Wide 部分
wide_output = Dense(1, activation='relu')(wide_inputs)

# 构建 Deep 部分
# 假设每个分类特征的嵌入维度为 4
embedding_feature4 = Embedding(input_dim=10, output_dim=4)(feature4_input)
embedding_feature5 = Embedding(input_dim=20, output_dim=4)(feature5_input)
embedding_feature6 = Embedding(input_dim=15, output_dim=4)(feature6_input)

# 合并嵌入层的输出，并添加额外的隐藏层
deep_output = concatenate([Flatten()(embedding_feature4), Flatten()(embedding_feature5), Flatten()(embedding_feature6)])
deep_output = Dense(64, activation='relu')(deep_output)
deep_output = Dense(32, activation='relu')(deep_output)

# 合并 Wide 和 Deep 部分的输出
combined = concatenate([wide_output, deep_output])

# 添加输出层
output = Dense(1, activation='sigmoid')(combined)

# 创建模型
model = Model(inputs=[wide_inputs, feature4_input, feature5_input, feature6_input], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 打印模型结构
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 feature4 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 feature5 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 feature6 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 1, 4)         40          ['feature4[0][0]']               
                                                                                            

In [11]:
# 训练模型
history = model.fit(
    train_ds,
    epochs=10,            # 指定训练的轮数
    validation_data=val_ds
)

# 评估模型
loss, accuracy = model.evaluate(val_ds)
print("Validation accuracy:", accuracy)


Epoch 1/10
250/250 [==============================] - 3s 3ms/step - loss: 0.6932 - accuracy: 0.5075 - val_loss: 0.6923 - val_accuracy: 0.5140
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 0.6928 - accuracy: 0.5110 - val_loss: 0.6923 - val_accuracy: 0.5160
Epoch 3/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6923 - accuracy: 0.5188 - val_loss: 0.6926 - val_accuracy: 0.5215
Epoch 4/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5261 - val_loss: 0.6941 - val_accuracy: 0.4980
Epoch 5/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6905 - accuracy: 0.5369 - val_loss: 0.6944 - val_accuracy: 0.5130
Epoch 6/10
250/250 [==============================] - 0s 2ms/step - loss: 0.6895 - accuracy: 0.5325 - val_loss: 0.6956 - val_accuracy: 0.4940
Epoch 7/10
250/250 [==============================] - 1s 2ms/step - loss: 0.6876 - accuracy: 0.5441 - val_loss: 0.6962 - val_accuracy: 0.5055
Epoch 

Wide & Deep 模型是一种由Google在2016年提出的深度学习模型，用于解决推荐系统等问题。该模型结合了线性模型（wide component）和深度模型（deep component）的优势，以提高模型在广义特征和深度特征之间进行有效学习的能力，从而更好地处理稀疏和高维数据。

**简介：**
- **Wide Component：** Wide 部分主要用于处理广义特征，这些特征通常是高度稀疏的，包含大量的类别型数据。Wide 部分采用线性模型，通过学习特征之间的交叉项，从而能够更好地捕捉广义特征之间的关联性。

- **Deep Component：** Deep 部分主要用于处理深度特征，这些特征通常是低维稠密的，包含连续型数据或低基数的类别型数据。Deep 部分采用深度神经网络，通过多层非线性变换来学习特征的抽象表示，从而能够更好地捕捉深度特征之间的复杂关系。

- **Wide & Deep 结合：** Wide & Deep 模型通过将 Wide 和 Deep 部分的输出连接在一起，通过一个全连接层进行融合，最终得到最终的预测结果。这种结合能够充分利用线性模型的记忆能力和深度模型的泛化能力，提高模型的整体性能。

**计算公式：**
Wide & Deep 模型的输出可以通过以下公式计算：

\[ \text{output} = \sigma\left(\text{wide\_component} + \text{deep\_component}\right) \]

其中，
- \(\text{wide\_component}\) 为 Wide 部分的线性模型输出。
- \(\text{deep\_component}\) 为 Deep 部分的神经网络输出。
- \(\sigma\) 通常为 sigmoid 函数，适用于二分类问题；或 softmax 函数，适用于多分类问题。

整个模型的训练过程是通过最小化损失函数来更新模型参数，损失函数通常为二分类交叉熵（binary cross-entropy）或多分类交叉熵（categorical cross-entropy）等。 Wide & Deep 模型在推荐系统等应用中表现出色，特别适用于同时处理广义和深度特征的任务。